In [1]:
import pandas as pd
from pathlib import Path
import re

In [2]:
big_path = "Traffic_Violations.csv"
big_df = pd.read_csv(big_path, low_memory=False)
path= "sample_data.csv"
df = pd.read_csv(path, low_memory=False)

In [3]:
df.head()

,SeqID,Date Of Stop,Time Of Stop,Agency,SubAgency,Description,Location,Latitude,Longitude,Accident,...,Charge,Article,Contributed To Accident,Race,Gender,Driver City,Driver State,DL State,Arrest Type,Geolocation
0,971ef50f-f138-419f-89e5-5d2cc5d7b75a,04/30/2023,23:55:00,MCP,"4th District, Wheaton",DRIVING VEHICLE ON HIGHWAY WITH SUSPENDED REGI...,KEMP MILL ROAD @ ALPERT LANE,39.053038,-77.024637,No,...,13-401(h),Transportation Article,False,WHITE,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(39.0530383333333, -77.0246366666667)"
1,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",NEGLIGENT DRIVING VEHICLE IN CARELESS AND IMPR...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-901.1(b),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
2,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",RECKLESS DRIVING VEHICLE IN WANTON AND WILLFUL...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-901.1(a),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
3,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",FAILURE TO DRIVE VEHICLE ON RIGHT HALF OF ROAD...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-301(a),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"
4,1b3ad74c-9e3b-47af-acba-cc3fcb656e6f,04/30/2023,04:24:00,MCP,"2nd District, Bethesda",DRIVING VEHICLE WHILE UNDER THE INFLUENCE OF A...,JONES MILL ROAD/ LE VELLE DRIVE,38.991480,-77.097113,No,...,21-902(a1i),Transportation Article,False,BLACK,F,SILVER SPRING,MD,MD,A - Marked Patrol,"(38.99148, -77.0971133333333)"


In [4]:
df.columns

Index(['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Driver City',
       'Driver State', 'DL State', 'Arrest Type', 'Geolocation'],
      dtype='object')

In [26]:
# Drop the columns that we have deemed unhelpful
big_df_smaller = big_df.drop(["Agency", "Geolocation", "VehicleType", "HAZMAT", 
                                "Commercial Vehicle", "Commercial License", "Article", "Charge", 
                                "SeqID", "SubAgency", "Model", "Contributed To Accident", "Driver State", 
                                "Driver City", "Arrest Type", "Search Reason For Stop"], axis = 1)

In [27]:
# Drop rows that do not result in a citation or warning
big_df_smaller.drop(big_df_smaller[big_df_smaller['Violation Type'] == 'ESERO'].index, inplace=True)
big_df_smaller.drop(big_df_smaller[big_df_smaller['Violation Type'] == 'SERO'].index, inplace=True)

In [28]:
# Clean the Year column
# Drop rows with null year
big_df_smaller = big_df_smaller.dropna(subset=["Year"])
# Change year to integer
big_df_smaller["Year"]= big_df_smaller["Year"].astype(int)
# Get years in a list
years = big_df_smaller["Year"].unique()
# Make a list of garbage years
years_to_remove = [0, 6338, 1005, 1196, 2912, 1009, 2088, 1007, 2102, 2109, 2105, 2997]
# Clean the list of years
good_years = [value for value in years if value not in years_to_remove]
# Keep only rows with good years
big_df_smaller = big_df_smaller[big_df_smaller["Year"].isin(good_years)]
big_df_smaller.shape

(1790375, 27)

In [29]:
# Keep only data that has latitude and longitude
big_df_smaller = big_df_smaller[(big_df_smaller['Latitude'] != 0) & (big_df_smaller['Longitude'] != 0)]


In [30]:
null_count = big_df_smaller['Accident'].isnull().sum()
null_count

0

In [31]:
if big_df_smaller.isnull().values.any():
    print("There are null values in the DataFrame.")
else:
    print("There are no null values in the DataFrame.")

There are null values in the DataFrame.


In [32]:
big_df_smaller["Search Reason"].value_counts(dropna = False)

NaN                       1594492
Incident to Arrest          38901
Probable Cause              19049
Consensual                  10615
K-9                          1463
Other                         873
Exigent Circumstances         440
Probable Cause for CDS          4
Arrest/Tow                      3
plain view marijuana            3
DUI                             1
Name: Search Reason, dtype: int64

In [8]:
print("Value counts for 'Search Reason' column for big_df:")
print(df['Search Reason'].value_counts())

print("\nValue counts for 'Search Reason' column for df:")
print(df['Search Reason'].value_counts())

Value counts for 'Search Reason' column for big_df:
Incident to Arrest    8
Consensual            2
Name: Search Reason, dtype: int64

Value counts for 'Search Reason' column for df:
Incident to Arrest    8
Consensual            2
Name: Search Reason, dtype: int64


In [78]:
big_df_smaller['Date Of Stop'] = pd.to_datetime(big_df_smaller['Date Of Stop'])
big_df_smaller['Year Date'] = big_df_smaller['Date Of Stop'].dt.year
big_df_smaller['Month'] = big_df_smaller['Date Of Stop'].dt.month
big_df_smaller['Day'] = big_df_smaller['Date Of Stop'].dt.day

In [34]:
# Drop rows with null Description
big_df_smaller = big_df_smaller.dropna(subset=["Description"])

In [35]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*EXCEEDING.*SPEED LIMIT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Speeding', x))

In [36]:
big_df_smaller["Description"].nunique()

14529

In [37]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAILURE.*YIELD.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Failure to Yield', x))

In [38]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAIL.*YIELD.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Failure to Yield', x))

In [39]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*LEARNER.*PERMIT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Learners Permit', x))

In [40]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*SPEED.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Speeding', x))

In [41]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*SPEEDING.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Speeding', x))

In [42]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*LAMP.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Equipment', x))

In [43]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*HEADLIGHT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Equipment', x))

In [44]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*REQUIRED.*MINIMUM.*EQUIPMENT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Equipment', x))

In [45]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*INOPERATIVE.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Equipment', x))

In [46]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*MISSING.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Equipment', x))

In [47]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*THROWING.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Littering', x))

In [48]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*DUMPING.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Littering', x))

In [49]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*PARK.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Non-Moving Violation', x))

In [50]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*ABANDONING.*VEH.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Non-Moving Violation', x))

In [51]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*INFLUENCE.*ALCOHOL.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'DUI', x))

In [52]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*IMPAIRED.*ALCOHOL.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'DUI', x))

In [53]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*DRUGS.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'DUI', x))

In [54]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*IMPAIRED.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'DUI', x))

In [55]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAILED.*STOP.*SIGN.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Stop', x))

In [56]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAILURE.*STOP.*SIGN.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Stop', x))

In [57]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAIL.*STOP.*SIGN.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Stop', x))

In [58]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAILURE.*STOP.*SIGNAL.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Stop', x))

In [59]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*FAILED.*STOP.*SIGNAL.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Improper Stop', x))

In [60]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*ELECTRONIC.*MSG.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Using Phone', x))

In [61]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*PHONE.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Using Phone', x))

In [62]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*TEXTMSG.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Using Phone', x))

In [63]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*TEXT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Using Phone', x))

In [64]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*TINT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Obstructed View', x))

In [65]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*OBSTRUCTED.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Obstructed View', x))

In [66]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*RECKLESS.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Reckless Driving', x))

In [67]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*ACCIDENT.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Involved in Accident', x))

In [68]:
# Use Regex to replace any description including speeding with just "Speeding"
pattern = re.compile(r'.*UNATTENDED.*DAMAGED.*', flags=re.IGNORECASE)

big_df_smaller['Description'] = big_df_smaller['Description'].apply(lambda x: re.sub(pattern, 'Involved in Accident', x))

In [69]:
df.columns

Index(['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Driver City',
       'Driver State', 'DL State', 'Arrest Type', 'Geolocation',
       'Search Reason Code', 'Violation Type Code', 'Search Outcome Code',
       'Search Disposition Type Code', 'Search Conducted Type Code'],
      dtype='object')

In [70]:
# Check for null values in big_df
big_df_null_counts = big_df.isnull().sum()
print("Null values in big_df:")
print(big_df_null_counts)

# Check for null values in df
df_null_counts = df.isnull().sum()
print("\nNull values in df:")
print(df_null_counts)

Null values in big_df:
SeqID                            0
Date Of Stop                     0
Time Of Stop                     0
Agency                           0
SubAgency                        0
Description                     10
Location                         4
Latitude                         0
Longitude                        0
Accident                         0
Belts                            0
Personal Injury                  0
Property Damage                  0
Fatal                            0
Commercial License               0
HAZMAT                           0
Commercial Vehicle               0
Alcohol                          0
Work Zone                        0
Search Conducted            716752
Search Disposition         1803841
Search Outcome              736010
Search Reason              1803841
Search Reason For Stop      717020
Search Type                1803849
Search Arrest Reason       1830456
State                           59
VehicleType                     

In [71]:
df_filled = df.fillna('Nothing')


In [72]:
print(df_filled.isnull().sum())

SeqID                           0
Date Of Stop                    0
Time Of Stop                    0
Agency                          0
SubAgency                       0
Description                     0
Location                        0
Latitude                        0
Longitude                       0
Accident                        0
Belts                           0
Personal Injury                 0
Property Damage                 0
Fatal                           0
Commercial License              0
HAZMAT                          0
Commercial Vehicle              0
Alcohol                         0
Work Zone                       0
Search Conducted                0
Search Disposition              0
Search Outcome                  0
Search Reason                   0
Search Reason For Stop          0
Search Type                     0
Search Arrest Reason            0
State                           0
VehicleType                     0
Year                            0
Make          

In [73]:
print("\nValue counts for 'Violation Type' column for df:")
print(df['Violation Type'].value_counts())


Value counts for 'Violation Type' column for df:
Citation    104
Warning      85
ESERO        10
Name: Violation Type, dtype: int64


In [75]:
# Define a mapping dictionary for the 'Violation Type'
violation_type_mapping = {'Citation': 1, 'Warning': 2, 'ESERO': 3}

# Map the values in the 'Violation Type' column to numeric values
df['Violation Type'] = df['Violation Type'].map(violation_type_mapping)


In [76]:
print("\nValue counts for 'Violation Type' column for df:")
print(df['Violation Type'].value_counts())


Value counts for 'Violation Type' column for df:
Series([], Name: Violation Type, dtype: int64)


In [77]:
big_df_smaller[['Fatal', 'Alcohol', 'Work Zone', 'Search Conducted',
       'Search Disposition', 'Search Outcome', 'Search Reason']].isnull()

,Fatal,Alcohol,Work Zone,Search Conducted,Search Disposition,Search Outcome,Search Reason
1,False,False,False,False,True,False,True
2,False,False,False,False,True,False,True
3,False,False,False,False,True,False,True
4,False,False,False,False,True,False,True
5,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...
1877994,False,False,False,False,True,False,True
1877996,False,False,False,True,True,True,True
1877999,False,False,False,False,True,False,True
1878000,False,False,False,False,True,False,True


In [83]:
print(df.columns)

Index(['SeqID', 'Date Of Stop', 'Time Of Stop', 'Agency', 'SubAgency',
       'Description', 'Location', 'Latitude', 'Longitude', 'Accident', 'Belts',
       'Personal Injury', 'Property Damage', 'Fatal', 'Commercial License',
       'HAZMAT', 'Commercial Vehicle', 'Alcohol', 'Work Zone',
       'Search Conducted', 'Search Disposition', 'Search Outcome',
       'Search Reason', 'Search Reason For Stop', 'Search Type',
       'Search Arrest Reason', 'State', 'VehicleType', 'Year', 'Make', 'Model',
       'Color', 'Violation Type', 'Charge', 'Article',
       'Contributed To Accident', 'Race', 'Gender', 'Driver City',
       'Driver State', 'DL State', 'Arrest Type', 'Geolocation'],
      dtype='object')


In [85]:
# Save the DataFrame with the new column to a new CSV file
df.to_csv('sample_data2.csv', index=False)